In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tqdm import tqdm

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

text = "Covid cases are increasing fast!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)
# text = "Covid cases are increasing fast!"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [ ]:
df = pd.read_csv('../input/stockmarket-sentiment-dataset/stock_data.csv')
df.head()

In [ ]:
pos_score = []
neu_score = []
neg_score = []

for index, text in tqdm(df.iterrows(), total=df.shape[0]):
# for text in tqdm(df['Text'].iterrows()):
    text = text[0]
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    neg_score.append(scores[0])
    neu_score.append(scores[1])
    pos_score.append(scores[2])
#     for i in range(scores.shape[0]):
#         l = config.id2label[ranking[i]]
#         s = scores[ranking[i]]
#         print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [ ]:
df.shape

In [ ]:
print(len(neg_score), len(neu_score), len(pos_score))

In [ ]:
# adding these values to the dataframe
df['neg_score'] = neg_score
df['neu_score'] = neu_score
df['pos_score'] = pos_score

In [ ]:
df.head()

In [ ]:
# if rounded off prediction of positive & neutral is equal to Sentiment column
hit = []
for s,p,n,neg in zip(df['Sentiment'], df['pos_score'], df['neu_score'], df['neg_score']):
    senti = round(p+n)
    if s == senti:
        hit.append(1)
    else:
        hit.append(0)
sum(hit)*100/len(hit)


## Using different dataset for Sentiment Analysis

In [ ]:
df_ = pd.read_csv('../input/stock-market-tweets-data-sentiment-analysis/tweets/tweets_labelled_09042020_16072020.csv',on_bad_lines='skip', sep = ';')
df_.head()

In [ ]:
df_.shape

In [ ]:
df_['sentiment'].unique()

In [ ]:
df_.dropna()

In [ ]:
def sentiment2int(word):
    if word == 'positive':
        return 1
    elif word == 'neutral':
        return 0
    else:
        return -1

In [ ]:
df_['sentiment_num'] = df_['sentiment'].apply(sentiment2int)
df_.head()

In [ ]:
pos_score = []
neu_score = []
neg_score = []

for index, text in tqdm(df_.iterrows(), total=df_.shape[0]):
# for text in tqdm(df['Text'].iterrows()):
    text = text[2]
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    neg_score.append(scores[0])
    neu_score.append(scores[1])
    pos_score.append(scores[2])
#     for i in range(scores.shape[0]):
#         l = config.id2label[ranking[i]]
#         s = scores[ranking[i]]
#         print(f"{i+1}) {l} {np.round(float(s), 4)}")

# adding these values to the dataframe
df_['neg_score'] = neg_score
df_['neu_score'] = neu_score
df_['pos_score'] = pos_score
df_.head()

In [ ]:
# if rounded off prediction of positive & neutral is equal to Sentiment column
hit = []
for s,p,n,neg in zip(df_['sentiment_num'], df_['pos_score'], df_['neu_score'], df_['neg_score']):
    senti_p = round(p)
    senti_neg = round(neg)
    senti_neu = round(n)
    if senti_p>senti_neg && 